In [1]:
import sys
sys.path.insert(0, '/home/anish/ASC_ML')

In [2]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [3]:
from ASC_ML import dataframe_extractor as de
from ASC_ML import model_generation as model_gen
from ASC_ML import model_gen_train_test as testing
from dask_ml.preprocessing import LabelEncoder
# from dask_ml.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

2022-06-03 11:04:50.463193: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
# import dask.dataframe as dd
# import pandas as pd
# data = dd.from_pandas(pd.Series(['a', 'a', 'b'], dtype='category'),
#                      npartitions=1)
# le = LabelEncoder()
# le.fit_transform(data).compute()
# print(type(le.fit_transform(data).compute()))

In [5]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"


# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1, label_names = ["SalePrice"]).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

Reading single csv from /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv
Train Dataset X Columns


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal


In [6]:
dataset_list[1].head()

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0


In [7]:
train_x = dataset_list[0].copy()

In [8]:
train_x = train_x.fillna(0)

In [9]:
train_x.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [10]:
train_x['MSSubClass'] = train_x['MSSubClass'].apply(str)

train_x['OverallCond'] = train_x['OverallCond'].astype(str)

train_x['YrSold'] = train_x['YrSold'].astype(str)
train_x['MoSold'] = train_x['MoSold'].astype(str)
train_x.head()

/home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/dask/dataframe/core.py:3858: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('MSSubClass', 'object'))

  warnings.warn(meta_warning(meta))


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [11]:
# a = train_x['LotShape']

In [12]:
# le = LabelEncoder() 
# le.fit_transform(a).compute()
# train_x['LotShape'] = le.fit_transform(train_x['LotShape'])

In [13]:
# train_x.head()

In [14]:
def keyval(allkeys):
    i = 0
    keyvalpairs = list()
    for key in allkeys:
        keyvalpairs.append((key,i))
        i+=1
    return dict(keyvalpairs)

In [15]:
# cols = ['FireplaceQu', 'LotShape']
cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
keys = {}
for col in cols:
    key = train_x[col].unique()
    keyvalpairs = keyval(key)
    keys.update({col:keyvalpairs})


In [16]:
keys

{'FireplaceQu': {0: 0, 'TA': 1, 'Gd': 2, 'Fa': 3, 'Ex': 4, 'Po': 5},
 'BsmtQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 0: 3, 'Fa': 4},
 'BsmtCond': {'TA': 0, 'Gd': 1, 0: 2, 'Fa': 3, 'Po': 4},
 'GarageQual': {'TA': 0, 'Fa': 1, 'Gd': 2, 0: 3, 'Ex': 4, 'Po': 5},
 'GarageCond': {'TA': 0, 'Fa': 1, 0: 2, 'Gd': 3, 'Po': 4, 'Ex': 5},
 'ExterQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'ExterCond': {'TA': 0, 'Gd': 1, 'Fa': 2, 'Po': 3, 'Ex': 4},
 'HeatingQC': {'Ex': 0, 'Gd': 1, 'TA': 2, 'Fa': 3, 'Po': 4},
 'PoolQC': {0: 0, 'Ex': 1, 'Fa': 2, 'Gd': 3},
 'KitchenQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'BsmtFinType1': {'GLQ': 0,
  'ALQ': 1,
  'Unf': 2,
  'Rec': 3,
  'BLQ': 4,
  0: 5,
  'LwQ': 6},
 'BsmtFinType2': {'Unf': 0,
  'BLQ': 1,
  0: 2,
  'ALQ': 3,
  'Rec': 4,
  'LwQ': 5,
  'GLQ': 6},
 'Functional': {'Typ': 0,
  'Min1': 1,
  'Maj1': 2,
  'Min2': 3,
  'Mod': 4,
  'Maj2': 5,
  'Sev': 6},
 'Fence': {0: 0, 'MnPrv': 1, 'GdWo': 2, 'GdPrv': 3, 'MnWw': 4},
 'BsmtExposure': {'No': 0, 'Gd': 1, 'Mn': 2, '

In [17]:
train_X = train_x.replace(keys).copy()

In [18]:
train_X['FireplaceQu'].unique().compute()

0    0
1    1
2    2
3    3
4    4
5    5
Name: FireplaceQu, dtype: int64

In [19]:
train_X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,0,RL,65.0,8450.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,0,WD,Normal
1,2.0,1,RL,80.0,9600.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,1,1,WD,Normal
2,3.0,0,RL,68.0,11250.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2,0,WD,Normal
3,4.0,2,RL,60.0,9550.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,2,WD,Abnorml
4,5.0,0,RL,84.0,14260.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,3,0,WD,Normal


In [20]:
cols_to_remove = []
for col in train_X.columns:
#     print(type(col))
    try:
        a = train_X[col].astype(float).compute()
    except ValueError:
#         print('Couldn\'t covert %s to float' % col)
        cols_to_remove.append(col)
        pass

# keep only the columns in df that do not contain string
train_X = train_X[[col for col in train_X.columns if col not in cols_to_remove]]

In [21]:
print(cols_to_remove)

['MSZoning', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']


In [22]:
train_X = train_X.drop(["Id"], axis = 1)

In [23]:
# tsklearn.preprocessing.MinMaxScalerrain_X.head()

In [24]:
len(train_X.columns)

58

In [25]:
len(dataset_list[1].columns)

1

In [26]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_X)
train_X_scaled = scaler_x.transform(train_X)

scaler_y = MinMaxScaler()
scaler_y.fit(dataset_list[1])
train_Y_scaled = scaler_y.transform(dataset_list[1])

In [27]:
# col = train_X.columns.to_list()
# scaler_d = MinMaxScaler()
# train_X.columns = list(train_X.columns)
# scaled_train_X = scaler_d.fit_transform(train_X[col])

In [28]:
train_X_scaled
# train_Y_scaled

array([[0.        , 0.20766773, 0.0334198 , ..., 0.        , 0.        ,
        0.        ],
       [0.07142857, 0.25559105, 0.03879502, ..., 0.        , 0.09090909,
        0.25      ],
       [0.        , 0.2172524 , 0.04650728, ..., 0.        , 0.18181818,
        0.        ],
       ...,
       [0.14285714, 0.21086262, 0.03618687, ..., 0.16129032, 0.09090909,
        1.        ],
       [0.07142857, 0.2172524 , 0.03934189, ..., 0.        , 0.63636364,
        1.        ],
       [0.07142857, 0.23961661, 0.04037019, ..., 0.        , 1.        ,
        0.        ]])

In [29]:
TRAIN = testing.Model_Parallel_Train_Test(train_X_scaled, train_Y_scaled, 58, 1, None)

In [30]:
TRAIN

In [31]:
model = TRAIN.get_models()
model.summary()

2022-06-03 11:05:09.270547: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-03 11:05:09.270927: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-03 11:05:09.316389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-03 11:05:09.316746: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.695GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-06-03 11:05:09.316835: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-03 11:05:10.435462: I tensorflow/stream_executor/platform/de

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layermodel_4n (InputLayer [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layermodel_8n (InputLayer [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layermodel_16n (InputLaye [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layermodel_32n (InputLaye [(None, 58)]         0                                            
______________________________________________________________________________________________

In [32]:
TRAIN.train_models()

2022-06-03 11:05:36.344291: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-03 11:05:36.365698: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2899885000 Hz


Epoch 1/100


2022-06-03 11:05:38.337259: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


12/12 [==============================] - 3s 7ms/step - loss: 0.1637 - output_layermodel_4n_loss: 0.0318 - output_layermodel_8n_loss: 0.0556 - output_layermodel_16n_loss: 0.0123 - output_layermodel_32n_loss: 0.0218 - output_layermodel_64n_loss: 0.0096 - output_layermodel_128n_loss: 0.0326 - output_layermodel_4n_mean_absolute_error: 0.1430 - output_layermodel_8n_mean_absolute_error: 0.2036 - output_layermodel_16n_mean_absolute_error: 0.0840 - output_layermodel_32n_mean_absolute_error: 0.1137 - output_layermodel_64n_mean_absolute_error: 0.0723 - output_layermodel_128n_mean_absolute_error: 0.1453
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0691 - output_layermodel_4n_loss: 0.0235 - output_layermodel_8n_loss: 0.0166 - output_layermodel_16n_loss: 0.0077 - output_layermodel_32n_loss: 0.0086 - output_layermodel_64n_loss: 0.0044 - output_layermodel_128n_loss: 0.0082 - output_layermodel_4n_mean_absolute_error: 0.1220 - output_layermodel_8n_mean_absolute_error: 0.09

12/12 [==============================] - 0s 5ms/step - loss: 0.0181 - output_layermodel_4n_loss: 0.0061 - output_layermodel_8n_loss: 0.0050 - output_layermodel_16n_loss: 0.0025 - output_layermodel_32n_loss: 0.0022 - output_layermodel_64n_loss: 0.0013 - output_layermodel_128n_loss: 0.0011 - output_layermodel_4n_mean_absolute_error: 0.0578 - output_layermodel_8n_mean_absolute_error: 0.0486 - output_layermodel_16n_mean_absolute_error: 0.0344 - output_layermodel_32n_mean_absolute_error: 0.0329 - output_layermodel_64n_mean_absolute_error: 0.0241 - output_layermodel_128n_mean_absolute_error: 0.0216
Epoch 15/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0188 - output_layermodel_4n_loss: 0.0059 - output_layermodel_8n_loss: 0.0053 - output_layermodel_16n_loss: 0.0027 - output_layermodel_32n_loss: 0.0023 - output_layermodel_64n_loss: 0.0014 - output_layermodel_128n_loss: 0.0011 - output_layermodel_4n_mean_absolute_error: 0.0565 - output_layermodel_8n_mean_absolute_error: 0.0

...

12/12 [==============================] - 0s 3ms/step - loss: 0.0096 - output_layermodel_4n_loss: 0.0032 - output_layermodel_8n_loss: 0.0028 - output_layermodel_16n_loss: 0.0018 - output_layermodel_32n_loss: 0.0011 - output_layermodel_64n_loss: 4.4668e-04 - output_layermodel_128n_loss: 1.8717e-04 - output_layermodel_4n_mean_absolute_error: 0.0393 - output_layermodel_8n_mean_absolute_error: 0.0357 - output_layermodel_16n_mean_absolute_error: 0.0287 - output_layermodel_32n_mean_absolute_error: 0.0233 - output_layermodel_64n_mean_absolute_error: 0.0145 - output_layermodel_128n_mean_absolute_error: 0.0094
Epoch 39/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0100 - output_layermodel_4n_loss: 0.0035 - output_layermodel_8n_loss: 0.0029 - output_layermodel_16n_loss: 0.0019 - output_layermodel_32n_loss: 0.0012 - output_layermodel_64n_loss: 3.8401e-04 - output_layermodel_128n_loss: 1.5956e-04 - output_layermodel_4n_mean_absolute_error: 0.0394 - output_layermodel_8n_mean_abs

12/12 [==============================] - 0s 4ms/step - loss: 0.0079 - output_layermodel_4n_loss: 0.0027 - output_layermodel_8n_loss: 0.0024 - output_layermodel_16n_loss: 0.0014 - output_layermodel_32n_loss: 8.6393e-04 - output_layermodel_64n_loss: 3.0247e-04 - output_layermodel_128n_loss: 1.2889e-04 - output_layermodel_4n_mean_absolute_error: 0.0354 - output_layermodel_8n_mean_absolute_error: 0.0332 - output_layermodel_16n_mean_absolute_error: 0.0251 - output_layermodel_32n_mean_absolute_error: 0.0209 - output_layermodel_64n_mean_absolute_error: 0.0125 - output_layermodel_128n_mean_absolute_error: 0.0085
Epoch 52/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0075 - output_layermodel_4n_loss: 0.0027 - output_layermodel_8n_loss: 0.0022 - output_layermodel_16n_loss: 0.0013 - output_layermodel_32n_loss: 7.4038e-04 - output_layermodel_64n_loss: 2.8317e-04 - output_layermodel_128n_loss: 1.8049e-04 - output_layermodel_4n_mean_absolute_error: 0.0349 - output_layermodel_8n_

12/12 [==============================] - 0s 4ms/step - loss: 0.0064 - output_layermodel_4n_loss: 0.0024 - output_layermodel_8n_loss: 0.0019 - output_layermodel_16n_loss: 0.0011 - output_layermodel_32n_loss: 6.5998e-04 - output_layermodel_64n_loss: 1.8481e-04 - output_layermodel_128n_loss: 6.1916e-05 - output_layermodel_4n_mean_absolute_error: 0.0324 - output_layermodel_8n_mean_absolute_error: 0.0304 - output_layermodel_16n_mean_absolute_error: 0.0238 - output_layermodel_32n_mean_absolute_error: 0.0184 - output_layermodel_64n_mean_absolute_error: 0.0094 - output_layermodel_128n_mean_absolute_error: 0.0056
Epoch 65/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0064 - output_layermodel_4n_loss: 0.0024 - output_layermodel_8n_loss: 0.0019 - output_layermodel_16n_loss: 0.0011 - output_layermodel_32n_loss: 6.8982e-04 - output_layermodel_64n_loss: 1.8939e-04 - output_layermodel_128n_loss: 5.4769e-05 - output_layermodel_4n_mean_absolute_error: 0.0321 - output_layermodel_8n_

12/12 [==============================] - 0s 4ms/step - loss: 0.0057 - output_layermodel_4n_loss: 0.0022 - output_layermodel_8n_loss: 0.0017 - output_layermodel_16n_loss: 9.8742e-04 - output_layermodel_32n_loss: 6.6171e-04 - output_layermodel_64n_loss: 1.1686e-04 - output_layermodel_128n_loss: 2.7018e-05 - output_layermodel_4n_mean_absolute_error: 0.0305 - output_layermodel_8n_mean_absolute_error: 0.0286 - output_layermodel_16n_mean_absolute_error: 0.0216 - output_layermodel_32n_mean_absolute_error: 0.0182 - output_layermodel_64n_mean_absolute_error: 0.0076 - output_layermodel_128n_mean_absolute_error: 0.0036
Epoch 78/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0058 - output_layermodel_4n_loss: 0.0022 - output_layermodel_8n_loss: 0.0018 - output_layermodel_16n_loss: 0.0011 - output_layermodel_32n_loss: 5.8248e-04 - output_layermodel_64n_loss: 1.1929e-04 - output_layermodel_128n_loss: 2.5580e-05 - output_layermodel_4n_mean_absolute_error: 0.0295 - output_layermodel

12/12 [==============================] - 0s 4ms/step - loss: 0.0051 - output_layermodel_4n_loss: 0.0020 - output_layermodel_8n_loss: 0.0014 - output_layermodel_16n_loss: 9.6607e-04 - output_layermodel_32n_loss: 5.1647e-04 - output_layermodel_64n_loss: 1.3123e-04 - output_layermodel_128n_loss: 2.3815e-05 - output_layermodel_4n_mean_absolute_error: 0.0289 - output_layermodel_8n_mean_absolute_error: 0.0266 - output_layermodel_16n_mean_absolute_error: 0.0209 - output_layermodel_32n_mean_absolute_error: 0.0153 - output_layermodel_64n_mean_absolute_error: 0.0080 - output_layermodel_128n_mean_absolute_error: 0.0034
Epoch 91/100
12/12 [==============================] - 0s 4ms/step - loss: 0.0045 - output_layermodel_4n_loss: 0.0016 - output_layermodel_8n_loss: 0.0015 - output_layermodel_16n_loss: 8.5379e-04 - output_layermodel_32n_loss: 5.0769e-04 - output_layermodel_64n_loss: 1.1706e-04 - output_layermodel_128n_loss: 2.1048e-05 - output_layermodel_4n_mean_absolute_error: 0.0272 - output_layerm